# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
# Imports here
import torch
from torch.optim import SGD, Adam
from torch.nn import CrossEntropyLoss, Sequential, Linear, ModuleList, ReLU, Dropout, ModuleList, Module,  LogSoftmax,NLLLoss, Conv2d
from torch import optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.datasets import ImageFolder
from torchvision.models import vgg19, densenet121, resnet18, resnet50, squeezenet1_1, alexnet, inception_v3
import torch.nn.functional 
from torch.nn.functional import relu
from collections import OrderedDict

In [2]:
def get_number_features_input(model, architecture = 'densnet'): 
    in_features = 0
    if architecture == 'resnet' or architecture == 'interceptron':        
        in_features = model.fc.in_features
    elif architecture == 'vgg': 
        in_features = model.classifier[6].in_features
    else:
        in_features = model.classifier.in_features
    return in_features

In [3]:
def print_freezee_layers(model):
    for name, child in model.named_children():
        for name_2, params in child.named_parameters():
            print(name_2, params.requires_grad)
    

In [4]:
def load_architecture(architecture = 'densnet'):
    model = None
    if architecture == 'resnet':        
        model = resnet18(pretrained = True)
    elif architecture == 'interceptron':
        model = inception_v3(pretrained = True)
    elif architecture == 'vgg': 
        model = vgg19(pretrained = True)
    else:
        model = densenet121(pretrained = True)
    return model

In [5]:
def freeze_features_layers(namelayer, model):
    ct = []
    for name, child in model.named_children():
        if namelayer in ct:
            for params in child.parameters():
                params.requires_grad = True
        ct.append(name)

In [6]:
def freeze_all_layers(model): 
    for i, param in model.named_parameters():
        param.requires_grad = False

In [7]:
def setup_outputlayer(new_architecture, model, architecture = 'densnet'):
    #Freeze all layers
    freeze_all_layers(model)
    
    #Setup output layer
    if architecture == 'resnet' or architecture == 'interceptron':        
        model.fc = new_architecture
        #Freeze only the features layer
        freeze_features_layers('fc', model)
    elif architecture == 'vgg': 
        features = list(model.classifier.children())
        features[6] = new_architecture
        model.classfier = Sequential(*features)
        model.num_classes = 102
        #Freeze only the features layer
        freeze_features_layers('classifier', model)
    else:
        model.classifier = new_architecture
        freeze_features_layers('classifier', model)
    
    #Print to verify the model
    print_freezee_layers(model)

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [8]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [9]:
def generate_transforms(dataset_type = 'train'): 
    transformation = None
    #Normalalize the values of RGB channels on the image.
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    if dataset_type == 'test': 
        transformation = transforms.Compose([
                                        transforms.RandomResizedCrop(224),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(),
                                        normalize
                                    ])
    else:
        transformation = transforms.Compose([
                                        transforms.RandomResizedCrop(224),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(),
                                        normalize
                                    ])
    return transformation

In [11]:
def load_trainsets(): 
    #Load images of train set
    train_set = ImageFolder(train_dir, transform = generate_transforms('train'))
    #Create an iterable of the train set.
    train_loader = DataLoader(train_set, batch_size = 64, shuffle = True)

    #Load images of test set
    test_set = ImageFolder(test_dir, transform = generate_transforms('test'))
    #Create an iterable of the test set.
    test_loader = DataLoader(test_set, batch_size = 64)

    #Load images of validation set
    validation_set = ImageFolder(valid_dir,  transform = generate_transforms('test'))
    #Create an iterable of the validation set.
    validation_loader = DataLoader(validation_set, batch_size = 64)
    
    print("***************************** TRAIN SET TRANSFORMATIONS ********************************")
    print(train_set)
    print("***************************** TEST SET TRANSFORMATIONS *********************************")
    print(test_set)
    print("***************************** VALIDATION SET TRANSFORMATIONS *********************************")
    print(test_set)
    return train_loader, test_loader, validation_set

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [12]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

In [13]:
def train_network(epochs, print_every, train_loader, criterion, optimizer, model, device = 'cpu'):
    #Steps 
    steps = 0
    #Loss calculated on running train set.
    running_loss = 0 
    #loss of the neural network
    loss = 0
    #loss on previous epochs
    previous_loss = 0

    model.to(device)

    #Iterate over the stpes. 
    for e in range(epochs):
        running_loss = 0 
        #Iterate over the batches
        for images, labels in train_loader: 
            steps += 1

            images, labels = images.to(device), labels.to(device)
            #Clear the weights accumulated
            optimizer.zero_grad()

            #feedforward
            prediction = model.forward(images)
            #loss
            loss = criterion(prediction, labels)
            #propagate error to the network 
            loss.backward()

            #Update weights 
            optimizer.step()

            running_loss += loss.item()

            if steps % print_every == 0: 
                print("Epoch: {}/{}... ".format(e+1, epochs),
                       "Loss: {:.4f}".format(running_loss/print_every))
                running_loss = 0

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [21]:
def validation(model, validationloader, criterion): 
    test_loss = 0
    accuracy = 0 
    loss = 0
    model.to('cpu')
    with torch.no_grad():        
        for images, labels in validationloader: 
            #Get the prediction calculated from the neural network
            prediction = model.forward(images)
            #Calculate loss betwix the prediction and real solution(labels). 
            loss += criterion(prediction, labels).item()
            print("Loss: "+str(loss))

            #Similarity of the output compared with the label. 
            equality = (labels.data == prediction.max(dim = 1)[1])
            accuracy += equality.type(torch.FloatTensor).mean()
            print("LABELS")
            print(labels)
            print("PREDICTION")
            print(prediction.max(dim = 1)[1])
            print("COMPARE")
            print(labels.data == prediction.max(dim = 1)[1])
            print("ACCURACY")
            print(accuracy)
    return accuracy, loss
    

In [15]:
def validation2(model, testloader, criterion):
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader: 
            images, labels = images.to('cuda'), labels.to('cuda')
            outputs = model(images)
            max_values, positions = torch.max(outputs.data, 1)
            total += labels.size(0)
            print("PREDICTION")
            print(positions)
            print("LABELS")
            print(labels)
            print("COMPARE")
            print(positions == labels)
            correct += (positions == labels).sum().item()
        print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

In [35]:
#Define the criterion that will calculate the loss of the prediction. 
criterion = NLLLoss()
#criterion = CrossEntropyLoss()
#Define the way of the batches will be passed to the network 
optimizer = Adam(list(filter(lambda p: p.requires_grad, model.parameters())), lr = 0.001)

NameError: name 'model' is not defined

In [16]:
#MAIN CELL 

architecture = 'vgg'
epochs = 2
print_every = 20

#Load data 
print("1. Stage: Load images")
trainloader, testloader, validationloader = load_trainsets()

#Define architecture of a pre trained network 
print("2. Stage define pre-trained model: "+architecture)
model = load_architecture(architecture)
print("---- Architecture")
print(model)

#Define the criterion that will calculate the loss of the prediction. 
print("3. Criterion and Optimizer")
criterion = NLLLoss()
print("---- NNLLoss")
print("---- Adam, lr = 0.001")
#criterion = CrossEntropyLoss()
#Define the way of the batches will be passed to the network 
optimizer = Adam(list(filter(lambda p: p.requires_grad, model.parameters())), lr = 0.001)

#Set up the output layer, to output only the necesary outputs to the specific problem 
input_size = get_number_features_input(model, architecture)
output_size = len(cat_to_name)
hidden_layers = [200, 150, 300]
layers =  Sequential(OrderedDict([
                          ('fc1', Linear(input_size, hidden_layers[0])),
                          ('relu_fc1', ReLU()),
                          ('fc2', Linear(hidden_layers[0], hidden_layers[1])),
                          ('relu_fc2', ReLU()),
                          ('fc3', Linear(hidden_layers[1], hidden_layers[2])),
                          ('relu_fc3', ReLU()),
                          ('fc4', Linear(hidden_layers[2], output_size)),
                          ('output', LogSoftmax(dim=1))
                          ]))
layers_vgg = Linear(input_size, output_size)
setup_outputlayer(layers, model, architecture)
print("4. Modified architecture.")
print(model)
print("--------- Freeze layers ")
print_freezee_layers(model)
#validation(model, testloader, criterion)
#validation2(model, testloader, criterion)


1. Stage: Load images
***************************** TRAIN SET TRANSFORMATIONS ********************************
Dataset ImageFolder
    Number of datapoints: 6552
    Root Location: flowers/train
    Transforms (if any): Compose(
                             RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
                             RandomHorizontalFlip(p=0.5)
                             ToTensor()
                             Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                         )
    Target Transforms (if any): None
***************************** TEST SET TRANSFORMATIONS *********************************
Dataset ImageFolder
    Number of datapoints: 819
    Root Location: flowers/test
    Transforms (if any): Compose(
                             RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
                             Rand

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.torch/models/vgg19-dcbb9e9d.pth
100%|██████████| 574673361/574673361 [00:18<00:00, 31734541.64it/s]


---- Architecture
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (

In [43]:
print(model)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [17]:
train_network(epochs, print_every, trainloader, criterion, optimizer, model, 'cuda')

Epoch: 1/2...  Loss: -4320.6064
Epoch: 1/2...  Loss: -50150.1264
Epoch: 1/2...  Loss: -174226.4355
Epoch: 1/2...  Loss: -419051.0187
Epoch: 1/2...  Loss: -761376.4594
Epoch: 2/2...  Loss: -1080425.4375
Epoch: 2/2...  Loss: -1870891.7500
Epoch: 2/2...  Loss: -2613405.9750
Epoch: 2/2...  Loss: -3531405.4125
Epoch: 2/2...  Loss: -4585512.1750


In [ ]:
validation(model, testloader, criterion)

Loss: -2927288.25
LABELS
tensor([  0,   0,   0,   0,   0,   1,   1,   1,   2,   2,   2,   2,
          2,   2,   2,   2,   3,   3,   3,   3,   4,   4,   4,   4,
          4,   4,   5,   5,   5,   5,   5,   5,   5,   5,   5,   6,
          6,   6,   6,   6,   6,   6,   6,   6,   7,   7,   7,   7,
          7,   7,   8,   8,   8,   9,   9,   9,   9,  10,  10,  10,
         11,  11,  11,  11])
PREDICTION
tensor([ 42,  42,  42,  42,  82,  41,  41,  42,  42,  42,  82,  42,
         82,  42,  44,  42,  42,  82,  82,  42,  82,  44,  42,  82,
         42,  42,  41,  82,  82,  42,  41,  44,  44,  42,  82,  42,
         42,  82,  44,  41,  42,  42,  82,  42,  41,  82,  42,  82,
         42,  44,  42,  42,  41,  82,  41,  44,  42,  41,  44,  41,
         82,  44,  42,  42])
COMPARE
tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0

In [18]:
# PRUEEEBA

# TODO: Define transforms for the training data and testing data
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

train_transforms =  transforms.Compose([
                                        transforms.RandomResizedCrop(224),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(),
                                        normalize,
                                        ])

test_transforms = transforms.Compose([
                                        transforms.RandomResizedCrop(224),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(),
                                        normalize,
                                    ])
# Pass transforms in here, then run the next cell to see how the transforms look
train_data = ImageFolder(train_dir, transform=train_transforms)
test_data = ImageFolder(test_dir, transform=test_transforms)

trainloader = DataLoader(train_data, batch_size=64, shuffle=True)
testloader = DataLoader(test_data, batch_size=32)

model = densenet121(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False
    
from collections import OrderedDict
classifier = Sequential(OrderedDict([
                          ('fc1', Linear(1024, 500)),
                          ('relu', ReLU()),
                          ('fc2', Linear(500, 2)),
                          ('output', LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier
#print_freezee_layers(model)

criterion = NLLLoss()
# Only train the classifier parameters, feature parameters are frozen
optimizer = Adam(model.classifier.parameters(), lr=0.001)

def train(epochs, trainloader, model, print_every, criterion, optimizer,  running_device='cpu'): 
    steps = 0
    running_loss = 0 
    
    model.to(running_device)
    for e in range(epochs):
        running_loss = 0
        for images, labels in trainloader: 
            steps += 1 
            
            images, labels = images.to(running_device), labels.to(running_device)
            
            optimizer.zero_grad()
            
            prediction = model.forward(images)
            loss = criterion(prediction, labels)
            loss.backward()

            optimizer.step()
            
            running_loss += loss.item()
            
            if steps % print_every == 0: 
                print("Epoch: {}/{}... ".format(e+1, epochs),
                      "Loss: {:.4f}".format(running_loss/print_every))
                running_loss = 0
                
                
def validation(testloader): 
    correct = 0 
    total = 0 
    with torch.no_grad(): 
        for images, labels in testloader: 
            images, labels = images.to('cuda'), labels.to('cuda')
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            print("PREDICTIONS")
            print(predicted)
            print("LABELS")
            print(labels)
            print("COMPARATION")
            print(predicted == labels)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
    

/opt/conda/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [12]:
# TODO: Save the checkpoint
def save_checkpoint(model, input_size, output_size):    
    model.class_to_idx = test_set.class_to_idx

    checkpoint = {
                    'input_size': input_size, 
                    'output_size': output_size,
                    'hidden_layers': [each.out_features for each in model.hidden_layers],
                    'state_dict': model.state_dict(), 
                    'class_ids': model.class_to_idx}

    torch.save(checkpoint, 'checkpoint.pth')

NameError: name 'image_datasets' is not defined

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = fc_model.Network(checkpoint['input_size'],
                             checkpoint['output_size'],
                             checkpoint['hidden_layers'])
    model.load_state_dict(checkpoint['state_dict'])
    
    return model

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes